In [140]:
import gmplot
import pandas as pd

In [49]:
if __name__ == "__main__": 
     
    file = open('C:\\Users\\S\\Downloads\\wimt_full_seperated_route_map_merged.txt',"r")
    
    gmap1 = gmplot.GoogleMapPlotter(21.1458,79.0882, 5 )  
    route_lat,route_lon = [],[]
    count,name = 0,""
    next(file)
    train_no = 1
    for line in file:  
        x = int(line.split('\t')[0])
        prev_name = name
        name = (line.split('\t')[1])
        
        if count == 0:
            count+=1
            train_no = int(x)
            try:
                route_lat.append(float(line.split('\t')[11]))
                route_lon.append(float(line.split('\t')[12][:-1]))
            except:
                continue #only for spaces
                
        else:
            if x == train_no:
                try:
                    route_lat.append(float(line.split('\t')[11]))
                    route_lon.append(float(line.split('\t')[12][:-1]))
                except:
                    continue #only for spaces
            else:
                train_no = int(x)
                
                gmap1.plot(route_lat, route_lon, color = 'blue', edge_width = 2)
                
                route_lat,route_lon = [],[]
                
                try:
                    route_lat.append(float(line.split('\t')[11]))
                    route_lon.append(float(line.split('\t')[12][:-1]))
                except:
                    continue #only for spaces
        
        count+=1       
        
    gmap1.plot(route_lat, route_lon, color = 'blue', edge_width = 2) 

In [145]:
import gmplot
import pandas as pd
import shapely
import math
from shapely.geometry import Point
from shapely.geometry import LineString
from math import radians, cos, sin, asin, sqrt


class Graph:
    
    def __init__(self):
        self.adjacency_list = dict()
        
    def add_edge(self,src,dest):
        if src not in self.adjacency_list:
            self.adjacency_list[src] = []
        self.adjacency_list[src].append(dest)
        
        if dest not in self.adjacency_list:
            self.adjacency_list[dest] = []
        self.adjacency_list[dest].append(src)
        
    def print_graph(self):
        for x in self.adjacency_list:
            print (x , "->" , self.adjacency_list[x])
            
    def find_degree(self,val):
        if val in self.adjacency_list:
            l = set(self.adjacency_list[val])
            return len(l)
        else:
            return 0

def filter_list(L):
    return [x for x in L if not any(set(x)<=set(y) for y in L if x is not y)]

def dist_bw2points(lat1,lon1,lat2,lon2):
    # to calculate Distance Between Two Points on Earth

    # converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    
    return (c*r)

def error_check(route_lat,route_lon,points,station_names,csv_file) :
    sum_error_dist,avg_err = 0,0
    line_joining_two_junctions = LineString([points[0],points[1]])
    #csv_file.write(station_names[0] + "," + '0' + '\n')
    for i in range(len(route_lat)): 
        point = Point(route_lat[i], route_lon[i])
        dist = line_joining_two_junctions.project(point)
        lis = list(line_joining_two_junctions.interpolate(dist).coords)
        err_dist = dist_bw2points(route_lat[i],route_lon[i],lis[0][0],lis[0][1])
        #csv_file.write(station_names[i+1] + "," + str(err_dist) + '\n')
        sum_error_dist += err_dist
        #csv_file.write(station_names[len(station_names)-1] +  "," + '0' + '\n')    
    if len(route_lat) > 0:
        avg_err = sum_error_dist / len(route_lat)
            
    csv_file.write(station_names[0] + "," + str(avg_err) + "," + station_names[len(station_names)-1]+ ","  + '\n')
   
    #csv_file.write(" " + "," + " " + "," + " " + '\n')
    return avg_err
    
if __name__ == "__main__":
    
    g = Graph()
    
    data = pd.read_csv('C:\\Users\\S\\Downloads\\station_info_aggregated.txt', sep='|')
    vertices = data['station_code'].tolist()
    
    data['lat'] = data['lat'].astype(float)
    data['lon'] = data['lon'].astype(float)
    
    latitudes = data['lat'].tolist()
    longitudes = data['lon'].tolist()
    
    data['popularity'] = data['popularity'].astype(int)
    popularity = data['popularity'].tolist()    
    
    file = open('C:\\Users\\S\\Downloads\\wimt_full_seperated_route_map_merged.txt',"r")
    count = 0
    next(file)
    train_no = 1
    name, prev_n = "",""
    for line in file:  
        x = int(line.split('\t')[0])
        prev_n = name
        name = line.split('\t')[1]
    
        if count == 0:
            count+=1
            train_no = int(x)
        else:
            if x == train_no:
                g.add_edge(name,prev_n)
            else:
                train_no = int(x)
        count+=1
       
    junc_endpoint = {}

    for i in range(len(vertices)):
        x = g.find_degree(vertices[i])
        if (x > 2 or x == 1) :
            junc_endpoint[vertices[i]] = i
  
    
    junc_to_junc_stations_list = []
    file.seek(0,0)
    points_names,stations_btw_two_junc = [],[]
    count,name = 0,""
    train_no = 1
    for line in file:  
        x = int(line.split('\t')[0])
        prev_name = name
        name = line.split('\t')[1]
        
        if count == 0:
            count+=1
            train_no = int(x)
            if name in junc_endpoint:
                points_names.append(name)
                stations_btw_two_junc.append(name)
        else:
            if x == train_no:
                if name in junc_endpoint:
                    points_names.append(name)
                    stations_btw_two_junc.append(name)
                    if len(points_names) == 2 :
                        l = stations_btw_two_junc.copy()
                        l.reverse()
                        if (stations_btw_two_junc not in junc_to_junc_stations_list) and (l not in junc_to_junc_stations_list) :
                            junc_to_junc_stations_list.append(stations_btw_two_junc)
                            
                        points_names,stations_btw_two_junc = [],[]
                        points_names.append(name)
                        stations_btw_two_junc.append(name)
                elif len(points_names) == 1:
                    stations_btw_two_junc.append(name)
                    
            else:
                
                train_no = int(x)
               
                points_names,stations_btw_two_junc = [],[]
                if name in junc_endpoint:
                    points_names.append(name)
                    stations_btw_two_junc.append(name)
                
                    
        count+=1 
        
    filter_list(junc_to_junc_stations_list)
    
    csv_file = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\avg_error_btw_junctions(indep_train)updated1.txt", "w")
    csv_file.write("First_junc_name" + "," + "Second_junc_name"  + "," + "err_dist" + '\n')
    
    #gmap1 = gmplot.GoogleMapPlotter(21.1458,79.0882, 5 )
    num_lines = 0
    for i in range(len(junc_to_junc_stations_list)) :
        points,routes_lat,routes_lon = [],[],[]
        lat,lon = [],[]
        for j in range(len(junc_to_junc_stations_list[i])):
            name = junc_to_junc_stations_list[i][j]
            if name in vertices:
                index = vertices.index(name)
                if j == 0 or j == len(junc_to_junc_stations_list[i])-1:
                    points.append((latitudes[index] , longitudes[index]))
                    lat.append(latitudes[index])
                    lon.append(longitudes[index])
                else:
                    routes_lat.append(latitudes[index])
                    routes_lon.append(longitudes[index])
                
        error_check(routes_lat, routes_lon,points,junc_to_junc_stations_list[i],csv_file) 
        num_lines +=1
        gmap1.scatter(lat,lon,'red', size = 100, marker = False)
        #gmap1.plot(lat,lon,color = 'yellow', edge_width = 2)
    gmap1.draw("C:\\Users\\S\\OneDrive\\Desktop\\removed_subsets.html")
    
    csv_file.close()
    print(num_lines)

1506


LTT VVH TNA
4.946966743940692
VVH TNA DIVA
51.44878880922302
TNA DIVA DTVL
7.856196338478142
DIVA DTVL PNVL
66.88485872073007
PNBE PWS DNR
9.32023283303576
PWS DNR ARA
1.872508848032851
DNR ARA DLN
2.399899427131018
ARA DLN JTHT
5.969021691982277
DLN JTHT DDU
12.878077956011822
JTHT DDU NEWC
0.6965403203036544
DDU NEWC AVCD
14.300287189243775
NEWC AVCD CAR
6.303746216082189
AVCD CAR MZP
40.793146316662714
CAR MZP KCN
9.496978334034239
MZP KCN PCOI
36.67496129010145
KCN PCOI LKJN
25.87070303108961
PCOI LKJN IDGJ
86.2804646658976
LKJN IDGJ MKP
40.80957251544528
IDGJ MKP BNSP
8.855457459868717
MKP BNSP STA
54.06185732128539
BNSP STA KTE
4.752635487819987
STA KTE KTES
28.650225377203014
KTE KTES KEQ
36.440401171719344
KTES KEQ ET
37.17543700103819
KEQ ET JUH
1.6160178360347857
ET JUH TLV
10.85547992233623
JUH TLV BSBN
27.503963428266403
TLV BSBN BSGS
30.944573427361036
BSBN BSGS JL
17.04278780275166
BSGS JL CSN
37.68724545179521
JL CSN MMR
23.59373273871131
CSN MMR YY-MMRX
1.68437996352093

BSB ZBD SLN
14.402797791222312
ZBD SLN UTR
15.999391236137974
SLN UTR LBH
54.853872753306845
UTR LBH LKO
81.68856514918967
LBH LKO LNWT
1.9818390032297657
LKO LNWT BLM
35.700002743776686
LNWT BLM ROZA
28.455338612755394
MBWC GJL HPU
10.091935823614719
GJL HPU GZB
2.3973500179863123
HPU GZB SBB
32.12005103438162
GZB SBB ANVT
44.89690838491333
SNT JUNC UDL
264.4840372126303
TDL WSC BRN
27.797342797193224
WSC BRN HRS
0.18875227284432405
BRN HRS ALJN
2.693001011477385
HRS ALJN KRJ
25.587503465644275
ALJN KRJ WAIR
1.4505027054184723
KRJ WAIR DKDE
0.06289144471840302
WAIR DKDE GZB
1.6268929141918065
KYQ DDNI NBQ
74.12459186446073
SMTA APDJ NMZ
1.8890203450273617
APDJ NMZ SGUJ
49.18888877931623
NMZ SGUJ NJP
260.19750715865484
SEE PMU CI
46.11363469375523
PMU CI CPR
3.402102029581471
CI CPR DDA
45.297973180261806
CPR DDA SV
2.391685605985103
DDA SV BTT
28.95569837975566
SV BTT GKC
13.769145067597023
GKP MUR GD
11.119014321537662
MUR GD BUW
31.479963135505415
GD BUW SCC
41.213819870504814
BRYC 

MALB DHG VGDC
24.099095745830937
AH MTJ KSJ
87.42760556207742
MTJ KSJ FBD
42.64695892031925
KSJ FBD CNB
25.632687663900427
NKE SGL MFP
6.563097023853857
XX-DKZO DKZ DBSI
21.79972870713513
DKZ DBSI PTNR
28.211826154139686
DBSI PTNR GHH
22.07473568339819
PTNR GHH XX-REO
8.461262823649903
GHH XX-REO RE
8.667234844329741
XX-REO RE RGS
6.282924950485892
RE RGS FL
19.16835982087227
BEC XX-BECD MVJ
20.527880023660998
XX-BECD MVJ UDZ
30.16677650731387
BKI XX-AWRO RE
6.9600436722267265
CLA SNCR JNJ
49.13289853921198
SNCR JNJ SWDV
40.26387523845719
JNJ SWDV BEPR
81.3496605597941
SWDV BEPR PNVL
11.006261779327318
GYJ XX-KLUO DMM
325.0717791009721
BYPA OML MGSJ
33.91625687684447
WADI WDC VKB
62.98798898198427
PRYJ PRRB MUV
5.1200396842592255
PRRB MUV BSB
70.0083285274008
IAA SRU BTT
37.685272280011574
PRYJ PRG PFM
61.98915462619407
PRG PFM PLP
83.97374072182282
PFM PLP JNH
7.216200662698711
PLP JNH XX-BSBC
18.701918354949584
JNH XX-BSBC BSB
40.59868655430711
MUE XX-MJRB MJRI
38.53746836975991
JNH 

In [141]:
import shapely
import math
from shapely.geometry import Point
from shapely.geometry import LineString
from math import radians, cos, sin, asin, sqrt

def dist_bw2points(lat1,lon1,lat2,lon2):
    # to calculate Distance Between Two Points on Earth

    # converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles

    return (c*r)        
        
def error_check(route_lat,route_lon,points,station_names,csv_file) :
    #csv_file.write(station_names[0]+ "," + "0" + '\n')
    sum_err_dist = 0
    for i in range(1,len(route_lat)-1):
        #if i == len(route_lat) -1:
            #csv_file.write(station_names[len(route_lat)-1]+ "," + "0" + '\n')
        #else:    
        point = Point(route_lat[i], route_lon[i])
        dist = LineString([points[0],points[1]]).project(point)
        l = list(LineString([points[0],points[1]]).interpolate(dist).coords)
        err_dist = dist_bw2points(route_lat[i],route_lon[i],l[0][0],l[0][1])
        sum_err_dist+=err_dist
            #csv_file.write(station_names[i]+ "," + str(err_dist) + '\n')
    csv_file.write(station_names[0]+ "," + str(sum_err_dist) + "," + station_names[len(route_lat)-1] + '\n')

def improve(route_lat,route_lon,err_dist,station_names,gmap1,csv_file):
    junc_lat,junc_lon,junc_names,i = [route_lat[0]],[route_lon[0]],[station_names[0]],0
    i = 1
    while i < len(err_dist):        
        while i < len(err_dist) and err_dist[i] >= 0 and err_dist[i] <= 20:
            i+=1  
        if i < len(err_dist) and err_dist[i] > 20:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])     
            junc_names.append(station_names[i])
        maxi1,index1 = 0,0    
        while i < len(err_dist) and err_dist[i] > 20 and err_dist[i] <= 40:
            if maxi1 < err_dist[i]:
                maxi1 = err_dist[i]
                index1 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 40:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index1>0:
            junc_lat.append(route_lat[index1])
            junc_lon.append(route_lon[index1])
            junc_names.append(station_names[i])
            
        maxi2,index2 = 0,0
        while i < len(err_dist) and err_dist[i] > 40 and err_dist[i] <= 60:
            if maxi2 < err_dist[i]:
                maxi2 = err_dist[i]
                index2 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 60:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index2>0:
            junc_lat.append(route_lat[index2])
            junc_lon.append(route_lon[index2])
            junc_names.append(station_names[i])
        
        maxi3,index3 = 0,0
        while i < len(err_dist) and err_dist[i] > 60 and err_dist[i] <= 80:
            if maxi3 < err_dist[i]:
                maxi3 = err_dist[i]
                index3 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 80:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index3>0:
            junc_lat.append(route_lat[index3])
            junc_lon.append(route_lon[index3])
            junc_names.append(station_names[i])
            
        maxi4,index4 = 0,0            
        while i < len(err_dist) and err_dist[i] > 80 and err_dist[i] <= 100:
            if maxi4 < err_dist[i]:
                maxi4 = err_dist[i]
                index4 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 100:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index4 > 0:
            junc_lat.append(route_lat[index4])
            junc_lon.append(route_lon[index4]) 
            junc_names.append(station_names[i])
            
    num_of_lines = 0        
    if i == len(err_dist):     
        junc_lat.append(route_lat[len(err_dist)-1])
        junc_lon.append(route_lon[len(err_dist)-1])
        junc_names.append(station_names[len(err_dist)-1])
        
        #gmap1.scatter(junc_lat,junc_lon,'red', size = 500, marker = False)
        
        j = 1
        points = []
        points.append((route_lat[0] ,route_lon[0]))
        start = 0
        for j in range(1,len(route_lat)):
            if (len(points) == 1) and (station_names[j] in junc_names):
                points.append((route_lat[j],route_lon[j]))
                end = j
                error_check(route_lat[start:end+1], route_lon[start:end+1],points,station_names[start:end+1],csv_file)
                points = []
                points.append((route_lat[j],route_lon[j]))
                start = j
        
        if len(points) == 2:
            end = j-1
            error_check(route_lat[start:end+1], route_lon[start:end+1],points,station_names[start:end+1],csv_file)
        if len(junc_lat) > 0:       
            gmap1.plot(junc_lat,junc_lon,color='yellow',edge_width = 2)
            num_of_lines = len(junc_lat)-1
        return num_of_lines
    
if __name__ == "__main__":
    
    csv_file = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\threshold_20_junc_to_jun_data.txt", "w")
    csv_file.write("station_code" + "," + "err_dist" + '\n')
    file = open('C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\error_dist_junc_to_junc_data.txt',"r")
    count,sr_no,name,route_lat,route_lon,station_names,err_dst = 0,0,"",[],[],[],[]
    next(file)
    name, prev_n = "",""
    sum_route_lines = 0
    for line in file:  
        x = int(line.split(',')[0])
        prev_n = name
        name = line.split(',')[1]
        lati,longi,err = float(line.split(',')[3]),float(line.split(',')[4][:-1]),float(line.split(',')[2])
        if count == 0:
            count+=1
            train_no = int(x)
            print(x)
            route_lat.append(lati)
            route_lon.append(longi)
            station_names.append(name)
            err_dst.append(err)
        else:
            if x == train_no:
                route_lat.append(lati)
                route_lon.append(longi)
                station_names.append(name)
                err_dst.append(err)
            else:
                train_no = int(x)
                #print("called")
                x = improve(route_lat,route_lon,err_dst,station_names,gmap1,csv_file)
                sum_route_lines += x
                route_lat, route_lon,err_dst,station_names = [],[],[],[]
                route_lat.append(lati)
                route_lon.append(longi)
                station_names.append(name)
                err_dst.append(err)
        count+=1 
    if len(route_lat) > 0:
        x = improve(route_lat,route_lon,err_dst,station_names,gmap1,csv_file)
        sum_route_lines += x
    file.close()
    csv_file.close()
    gmap1.draw( "C:\\Users\\S\\OneDrive\\Desktop\\improved_only_new_routes(threshold_20).html" )
    print(sum_route_lines)

0
1669


In [143]:
def GetAngle (points):
    lat1, lon1 = points[0][0],points[0][1]
    lat2, lon2 = points[1][0],points[1][1]
    dX = lat2 - lat1
    dY = lon2 - lon1
    rads = math.atan2(-dY, dX) 
    return math.degrees(rads)

if __name__ == "__main__":
    csv_file = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\threshold_20_junc_to_jun_data.txt", "r")
    csv_file1 = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\joining_junc_to_jun_data.txt", "w")
    csv_file1.write("first_point" + "," + "second_point" + "," + "thrid_point" + "," + "angle" + '\n')
    next(csv_file)
    count = 0
    first,second = "",""
    joining_junc_to_junc_data=[]
    points,prev_points,l=[],[],[]
    for line in csv_file:  
        prev_l = l
        prev_first = first
        first = line.split(',')[0]
        prev_sec = second
        second = line.split(',')[2][:-1]
        prev_points = points
        points = []
        l = [first,second]
        
        
        if count == 0:
            count+=1
            index1 = vertices.index(first)
            index2 = vertices.index(second)
            points.append((latitudes[index1] , longitudes[index1]))
            points.append((latitudes[index2] , longitudes[index2]))
           
        else:

            if (first in vertices) and (second in vertices):
                index1 = vertices.index(first)
                index2 = vertices.index(second)
                points.append((latitudes[index1] , longitudes[index1]))
                points.append((latitudes[index2] , longitudes[index2]))

            if first == prev_sec:
                theta1 = GetAngle(points)
                theta2 = GetAngle(prev_points)
                ang = abs(theta2 - theta1)
                
                csv_file1.write(prev_first + "," + first + "," + second + "," + str(ang) + '\n')
                
                if prev_l not in joining_junc_to_junc_data:
                    joining_junc_to_junc_data.append(prev_l)
                if l not in joining_junc_to_junc_data:
                    joining_junc_to_junc_data.append(l)
                #print(ang)
                
    csv_file.close()
    csv_file1.close()
    print(len(joining_junc_to_junc_data))
    
    

1248


In [117]:
csv_file = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\joining_junc_to_jun_data.txt", "r")
next(csv_file)
num_lines,count = 0,0
junc_already_covered,lat,lon,l1,l2,angle = [],[],[],[],[],""
for line in csv_file:  
    prev_l1,prev_l2,prev_angle=l1,l2,angle
    angle = line.split(',')[3][:-1]
    first,second,third = line.split(',')[0],line.split(',')[1],line.split(',')[2]
    index1,index2,index3 = vertices.index(first),vertices.index(second),vertices.index(third)
    l1,l2 = [first,second],[second,third]
    if float(angle) <= 12:
        if count == 0:
            count+=1
            lat = [latitudes[index1],latitudes[index3]]
            lon = [longitudes[index1],longitudes[index3]]
        else:
            if prev_l2 == l1 and float(prev_angle) <= 12:
                lat[1] = latitudes[index3]
                lon[1] = longitudes[index3]
            else:
                if len(lat) > 0:
                    num_lines+= 1
                    gmap1.plot(lat,lon,color = 'yellow', edge_width = 2)
                lat,lon = [],[]
                lat = [latitudes[index1],latitudes[index3]]
                lon = [longitudes[index1],longitudes[index3]]
            
        junc_already_covered.extend([l1,l2])
        
      
if len(lat) > 0:
    num_lines+= 1
    gmap1.plot(lat,lon,color = 'yellow', edge_width = 2)
lat,lon = [],[]
    
csv_file.seek(0,0)    
next(csv_file)
for line in csv_file:  
    angle = line.split(',')[3][:-1]
    first,second,third = line.split(',')[0],line.split(',')[1],line.split(',')[2]
    index1,index2,index3 = vertices.index(first),vertices.index(second),vertices.index(third)
    l1,l2 = [first,second],[second,third]
    if float(angle) > 12:
        if l1 not in junc_already_covered:
            lat.extend([latitudes[index1],latitudes[index2]])
            lon.extend([longitudes[index1],longitudes[index2]])
        if l2 not in junc_already_covered:
            lat.extend([latitudes[index2],latitudes[index3]])
            lon.extend([longitudes[index2],longitudes[index3]])  
    if len(lat) > 0:
        num_lines+= len(lat)-1
        gmap1.plot(lat,lon,color = 'yellow', edge_width = 2)
    lat,lon = [],[]

    
csv_file1 = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\threshold_20_junc_to_jun_data.txt", "r")
next(csv_file1)
for line in csv_file1:  
    first = line.split(',')[0]
    second = line.split(',')[2][:-1]
    l = [first,second]
    if l not in joining_junc_to_junc_data:
        lat,lon = [],[]
        index1,index2 = vertices.index(l[0]),vertices.index(l[1])
        lat.extend([latitudes[index1],latitudes[index2]])
        lon.extend([longitudes[index1],longitudes[index2]])
        if len(lat) > 0:
            num_lines+= 1
            gmap1.plot(lat,lon,color = 'yellow', edge_width = 2)
    
gmap1.draw("C:\\Users\\S\\OneDrive\\Desktop\\improved_only_new_routes(threshold_20)updated.html" )  
print(num_lines)

#print(junc_already_covered)

2277


0
1826


In [104]:
data = pd.read_csv('C:\\Users\\S\\Downloads\\station_info_aggregated.txt', sep='|')
v = data['station_code'].tolist()
data1 = pd.read_csv('C:\\Users\\S\\Downloads\\station_info_with_pop.txt', sep='|')
v1 = data1['station_code'].tolist()
l = list(set(v) - set(v1))
print(l)

['XXXX', 'ROH', 'XX-URKO', 'CP', 'XX-SCB', 'VGRA', 'SB04', 'BGDA', 'XX-RVH', 'BRKD', 'JYD', 'GHAT', 'XX-AJIC', 'NMA', 'XX-YPRB', 'XX-UJNO', 'XX-DTVC', 'SDNI', 'MHANA', 'GPL', 'SNTN', 'XX-QLNC', 'NRKG', 'SLQ', 'BHEG', 'KSJF', 'YY-CPHB', 'CHKB', 'XX-WKRC', 'XX-MCJO', 'XX-AIIO', 'JHDC', 'XX-SLRO', 'BHAT(B)', 'BLYC', 'ARMR', 'GY-DLS', 'GOPL', 'DHCA', 'QSBA', 'XX-KGPO', 'XX-NPTB', 'LFG', 'BRGNP', 'EPR', 'SUNC', 'VLKN', 'SLTNP', 'SRHRI', 'JTG', 'KLLRD', 'LMI', 'PBY', 'MCR', 'XX-TENR', 'RVSN', 'BF', 'XX-TOKB', 'LTNR', 'NKKH', 'PKJN', 'TJN', 'MHOW', 'YY-TBC', 'XX-MQTRACK', 'DDAC', 'OSHI', 'VNEC', 'DA', 'SRS', 'EC09', 'BTKA', 'PURA', 'JNPD', 'MNBG', 'DNC', 'FHT', 'XX-RPEC', 'TLB(B)', 'GHF', 'BNHL', 'SXH', 'XX-ANNO', 'HTGA', 'ASZ', 'YY-SBGB', 'RAS', '0PS-R', 'AKA', 'XX-KJMO', 'TKV', 'MMDPL', 'KQC', 'UMNF', 'RCT', 'LSGH', 'XX-BNDB', 'NKT', 'HPX', 'XX-LKOB', 'BPI', 'FNI', 'STK', 'BANIKPUR', 'NCHT', 'YY-NTSS', 'CSTM', 'KDRL', 'SNPD', 'FJT', 'GGX', 'BYI', 'CML', 'TPNR', 'BRGZ', 'MD', 'UMA', 'XX-NFKB

In [144]:
csv_file = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\joining_junc_to_jun_data.txt", "r")
next(csv_file)
num_lines,count = 0,0
btw_points = {}
junc_already_covered,lat,lon,l1,l2,angle = [],[],[],[],[],""
for line in csv_file:  
    prev_l1,prev_l2,prev_angle=l1,l2,angle
    angle = line.split(',')[3][:-1]
    first,second,third = line.split(',')[0],line.split(',')[1],line.split(',')[2]
    index1,index2,index3 = vertices.index(first),vertices.index(second),vertices.index(third)
    l1,l2 = [first,second],[second,third]
    if float(angle) <= 12:
        if count == 0:
            count+=1
            lat = [latitudes[index1],latitudes[index3]]
            lon = [longitudes[index1],longitudes[index3]]
            btw_points[index2] = [latitudes[index2],longitudes[index2]]
        else:
            if prev_l2 == l1 and float(prev_angle) <= 12:
                ind = latitudes.index(lat[1])
                lat[1] = latitudes[index3]
                lon[1] = longitudes[index3]
                btw_points[ind] = [latitudes[ind],longitudes[ind]]
            else:
                if len(lat) > 0:
                    num_lines+= 1
                    gmap1.plot(lat,lon,color = 'yellow', edge_width = 2)
                    coord = []
                    print(btw_points)
                    if len(btw_points) > 0:
                        print("go")
                        for x in btw_points:
                            point = Point(btw_points[x][0],btw_points[x][1])
                            l =  LineString([(lat[0],lon[0]),(lat[1],lon[1])])
                            dist = l.project(point)
                            cd = list(l.interpolate(dist).coords)
                            coord.append(cd)
                            latitudes[x] = cd[0][0]
                            longitudes[x] = cd[0][1]
                        coord_lat = [i[0][0] for i in coord]
                        coord_lon = [i[0][1] for i in coord]      
                        gmap1.scatter(coord_lat,coord_lon,'green', size = 500, marker = False)       
                lat,lon,coord = [],[],[]
                btw_points.clear()
                lat = [latitudes[index1],latitudes[index3]]
                lon = [longitudes[index1],longitudes[index3]]
                btw_points[index2] = [latitudes[index2],longitudes[index2]]
            
        junc_already_covered.extend([l1,l2])
        
      
if len(lat) > 0:
    num_lines+= 1
    gmap1.plot(lat,lon,color = 'yellow', edge_width = 2)
lat,lon = [],[]
    
csv_file.seek(0,0)    
next(csv_file)
for line in csv_file:  
    angle = line.split(',')[3][:-1]
    first,second,third = line.split(',')[0],line.split(',')[1],line.split(',')[2]
    index1,index2,index3 = vertices.index(first),vertices.index(second),vertices.index(third)
    l1,l2 = [first,second],[second,third]
    if float(angle) > 12:
        if l1 not in junc_already_covered:
            lat.extend([latitudes[index1],latitudes[index2]])
            lon.extend([longitudes[index1],longitudes[index2]])
        if l2 not in junc_already_covered:
            lat.extend([latitudes[index2],latitudes[index3]])
            lon.extend([longitudes[index2],longitudes[index3]])  
    if len(lat) > 0:
        num_lines+= len(lat)-1
        gmap1.plot(lat,lon,color = 'yellow', edge_width = 2)
    lat,lon = [],[]

    
csv_file1 = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\threshold_20_junc_to_jun_data.txt", "r")
next(csv_file1)
for line in csv_file1:  
    first = line.split(',')[0]
    second = line.split(',')[2][:-1]
    l = [first,second]
    if l not in joining_junc_to_junc_data:
        lat,lon = [],[]
        index1,index2 = vertices.index(l[0]),vertices.index(l[1])
        lat.extend([latitudes[index1],latitudes[index2]])
        lon.extend([longitudes[index1],longitudes[index2]])
        if len(lat) > 0:
            num_lines+= 1
            gmap1.plot(lat,lon,color = 'yellow', edge_width = 2)
    
gmap1.draw("C:\\Users\\S\\OneDrive\\Desktop\\improved_only_new_routes(threshold_20)updated_with_projections.html" )  
print(num_lines)

#print(junc_already_covered)

{1472: [19.078579173850407, 72.89797109880433]}
go
{592: [19.186783674732688, 73.0424576482541]}
go
{1730: [25.59422412886361, 85.07946363822792], 92: [25.5885515111906, 85.04267538598937], 395: [25.530251853370057, 84.6645884707494], 697: [25.37761770844476, 83.67472034506089]}
go
{10131: [25.278695690498726, 83.11979182082668], 8707: [25.273229999999998, 83.10879], 8088: [25.254224629720156, 83.08230791481219]}
go
{495: [25.168526237254284, 82.57636360941731]}
go
{222: [25.067984169431494, 81.09242374284047]}
go
{270: [24.580744035076968, 80.80407365752437]}
go
{161: [22.42765134801309, 77.83174158838358], 7096: [22.424303164724044, 77.82244910480776]}
go
{5390: [21.399804164294608, 76.25006675717616]}
go
{147: [20.24993455193117, 74.43805390454087]}
go
{2480: [19.21127336052295, 73.07701835104635], 8325: [19.196848962930837, 73.05666213568256]}
go
{547: [25.542895715655114, 83.18980022776053]}
go
{8754: [23.279534471964897, 77.42362701519687]}
go
{2647: [20.151612939339714, 74.47205

{5162: [23.45758034318498, 72.44253280287825], 1896: [23.25059124997452, 72.51918464364996], 2785: [23.15787854792469, 72.55351785012385]}
go
{700: [26.774330645311775, 84.75609707023932]}
go
{1209: [28.457485903504015, 76.91463087576864], 9914: [28.210162810148685, 76.61885625781474]}
go
{4473: [27.9601090552406, 76.02018450360437], 1208: [27.355025170726005, 75.58907974672206]}
go
{9828: [27.579594163918514, 76.58416895656948]}
go
{243: [19.016048847698166, 73.03826825203754]}
go
{10487: [25.442515941191598, 81.85145075470197]}
go
{1242: [25.537941468519886, 82.09193962933436]}
go
{130: [25.912901, 82.001535], 2120: [25.94650845385845, 82.00673692042099], 136: [26.266369007180145, 82.05624644742058]}
go
{1202: [21.739799903560193, 70.55017589439949]}
go
{2877: [23.74242140860985, 86.82541197321403]}
go
{8361: [17.1627173553905, 75.07109812579183], 2985: [16.895276141710184, 74.90022383521695]}
go
{5229: [24.20680298587908, 83.31060010299491]}
go
{2337: [23.8308569263446, 84.195214628

In [134]:
point = Point(12,24)
dist = LineString([(15,30),(12,24)]).project(point)
l = list(LineString([points[0],points[1]]).interpolate(dist).coords)
print(l)

[(19.010663, 73.024721)]
